# CHƯƠNG 2: QUY TRÌNH XÂY DỰNG MỘT HỆ THỐNG HỌC MÁY - PHẦN 2
----

* Mã sinh viên:
* Họ và tên:
* Lớp:
* Email:
---

### 3.3 MỘT SỐ KỸ THUẬT TIỀN XỬ LÝ DỮ LIỆU
---


**XỬ LÝ TẬP DỮ LIỆU TITANIC:**

Tập dữ liệu Data_Titanic.csv bao gồm: 1309 bản ghi tương ứng với 1309 hành khách (với đầy đủ các thông tin liên quan)

Mỗi hành khách (mẫu) bao gồm các thuộc tính:
1.  PassengerId: ID của hành khách
2.  Survival: Thuộc tính cho biết hành khách được cứu hay không (ground truth) 0 = No (Không được cứu); 1 = Yes (Được cứu)
3.   Pclass: Loại vé tàu (Cho biết Địa vị xã hội): 1 - Hạng nhất; 2 - Hạng hai; 3 - Hạng 3
4.   Name: Tên của hành khách
5.  Sex: Giới tính (Male/Female)
6. Age: Tuổi (Tính theo năm)
7. SibSp: Số lượng anh chị em/bạn đời trên khoang tàu 
8. Parch: Số lượng cha mẹ/con cái trên khoang tàu
9. Ticket: Số hiệu vé
10. Fare: Phí hành khách
11. Cabin: Số hiệu cabin
12. Embarked: Cổng lên tàu (C = Cherourg; Q = Queenstown, S=Southampton)

In [2]:
#Gọi các thư viện sử dụng cho Project:

# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### 1) Tải tập dữ liệu:
---

In [3]:
path_titanic ="Data/Data_Titanic.csv"
df_titanic = pd.read_csv(path_titanic)
df_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     1309 non-null   int64  
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 122.8+ KB


In [ ]:
#Hiển thị danh sách thông tin 10 hành khách đầu tiên
df_titanic.head(10)

In [ ]:
#Hiển thị dữ liệu 10 hành khách cuối cùng
df_titanic.tail(10)

### 2) Phân tích - Thống kê số liệu 
---

In [ ]:
#Xây dựng hàm thống kê dữ liệu missing trong dataframe:
#-----------------------------------------------------
#Đầu vào của hàm là 1 biến Dataframe
#Đầu ra bao gồm các thông số:
#Tổng số cột của file dữ liệu
#Tổng số cột có chứa dữ liệu missing
#Danh sách các cột chứa dữ liệu missing với 2 thống số:
#Tổng số giá trị missing tương ứng với cột đó
#Tỷ lệ % dữ liệu missing trên tổng số dữ liệu của cột
def missing_values(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Số giá trị Missing', 1 : 'Tỷ lệ % missing'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        'Tỷ lệ % missing', ascending=False).round(2)
        print ("File dữ liệu bao gồm có: " + str(df.shape[1]) + " cột.\n"      
            "Có " + str(mis_val_table_ren_columns.shape[0]) +
              " cột chứa missing values.")
        return mis_val_table_ren_columns

In [ ]:
print('Số lượng missing data trong tập Train: \n')
missing_values(df_titanic)

In [ ]:
#Phân tích số liệu tập Train
df_titanic.describe()

In [ ]:
#Thông kê tập dữ liệu Train các thuộc tính có dtype: Object
df_titanic.describe(include=['O'])

## Phân tích các thuộc tính quan trọng ảnh hưởng đến khả năng sống/chết (survived) của hành khách:
---
Mỗi một hành khách bao gồm 12 thuộc tính (tương ứng với 12 cột dữ liệu)



**a) Thuộc tính Hạng vé tàu - Pclass:**
---
Pclass: Loại vé tàu của hành (cho biết địa vị xã hội) 1 – Hạng nhất | 2 – Hạng 2 | 3 – Hạng ba

In [ ]:
#Đặc tính Hạng vé tàu - Pclass:
df_titanic[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

**Pclass: ảnh hưởng tới khả năng được cứu của hành khách. Hành khách có vé hạng nhất khả năng được cứu cao hơn hạng 2,3**

**b) Thuộc tính Giới tính - Sex:**
---

In [ ]:
#Đặc tính Giới tính - Sex:
df_titanic[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

**Sex: Ảnh hưởng cao tới khả năng được cứu của hành khách. Hành khách là Nữ (female) có tỷ lệ được cứu sống là ~74% trong khi hành khách Nam chỉ có tỷ lệ cứu sống là ~19%**

**c) Thuộc tính tuổi - Age:**
---

In [ ]:
plt.hist(df_titanic.Age,bins=20)
plt.grid(True)
plt.show()

In [ ]:
#Thống kê liệu tuổi hành khách còn sống|chết theo tuổi
g = sns.FacetGrid(df_titanic, col='Survived')
g.map(plt.hist, 'Age', bins=40)

Quan sát dữ liệu Tuổi:
*  Hầu hết hành khách đi tàu nằm trong độ tuổi từ 15 - 35
*  Trẻ em (Tuổi <=4) có tỷ lệ được cứu cao
*  Các hành khách già nhất (Tuổi = 80) đều được cứu
*  Một lượng lớn hành khách trong độ tuổi từ 15 - 25 không được cứu

Xem xét:

*  Thuộc tính Age ảnh hưởng tới khả năng được cứu của hành khách --> đưa vào mô hình huấn luyện
*  Xử lý một số dữ liệu missing của thuộc tính age
*  Nhóm các độ tuôi theo từng group

**d) Thuộc tính SibSp:**
---
SibSp (Siblings/Spouses): Số lượng anh chị em| vợ chồng của hành khách cùng trên tàu(Kiểu dữ liệu: số)

In [ ]:
df_titanic[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

**SibSp: Có ảnh hưởng tới khả năng được cứu của hành khách. Những người có 1,2 người thân đi cùng hoặc đi một mình có tỷ lệ được cứu cao hơn những người khác**

**e) Thuộc tính Parch:**
---
Parch(Parents/Children): Số lượng bố mẹ| con cái của hành khách cùng trên tàu(Kiểu dữ liệu: số)


In [ ]:
df_titanic[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

**f) Thuộc tính Fare:**
---
Fare: Giá vé đi tàu của hành khách (Kiểu dữ liệu: Số)

In [ ]:
plt.hist(df_titanic.Fare,bins=40)
plt.grid(True)
plt.show()

In [ ]:
#Thống kê liệu tuổi hành khách còn sống|chết theo tuổi
g = sns.FacetGrid(df_titanic, col='Survived')
g.map(plt.hist, 'Fare', bins=40)

**g) Thuộc tính Embarked:**
---
Ký hiệu cho biết cảng mà hành khách lên tàu, Tàu Titanic đón khách ở 3 cảng: C = Cherbourg | Q = Queenstown | S =Southampton (Kiểu dữ liệu: ký tự)


In [ ]:
#Đặc tính Giới tính - Sex:
df_titanic[["Embarked", "Survived"]].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

**Thuộc tính Embarked có ảnh hưởng tới khả năng được cứu, những hành khách lên tàu ở cảng S, Q tỷ lệ được cứu cao hơn là lên tàu ở cảng S**

## 3) Tiền xử lý dữ liệu
---

#### 3.1) Loại bỏ các thuộc tính không quan trọng
---
* Một số thuộc tính không|ít ảnh hưởng tới việc hành khách được cứu hay không, cần loại bỏ những thuộc tính này khỏi dữ liệu. Để giảm chiều dữ liệu và nâng cao chất lượng của dữ liệu huấn luyện.

* Có 4 thuộc tính không ảnh hưởng|ít ảnh hưởng bao gồm:

    1. **PassengerId:** Thuộc tính này chỉ mà id, đánh số thứ tự của hành khách (Bỏ)
    2. **Name:** Tên của hành khách --> (Bỏ)
    3. **Ticket:** Số vé của hành khách (Bỏ)
    4. **Fare:** Giá vé - Giá vé đã được phản ánh thông qua thuộc tính Pclass (hạng vé: 1, 2, 3) (Bỏ)
    5. **Cabin:** Số hiệu Cabin trên tàu của hành khách - Thuộc tính này có tỷ lệ dữ liệu thiếu rất lớn, chiếm ~77,5% dữ liệu --> Bỏ

In [ ]:
df_titanic.describe(include='O')

In [ ]:
#Tạo một DataFrame mới df_data loại bỏ đi 5 thuộc tính ko quan trọng
df_data = df_titanic.drop(['PassengerId','Name','Ticket','Fare', 'Cabin'], axis=1)
df_data.info()

#### 3.2) Xử lý các thuộc tính missing value:
---
* Các mô hình học máy không làm việc được với các bản ghi chứa thuộc tính missing do đó cần phải được xử lý.

* Trong tập dữ liệu hiện tại có thuộc tính chứa giá trị missing, bao gồm:

    1. **Age:** Missing 263 bản ghi (chiếm 20.09%)
    2. **Embarked:** Missing 2 bản ghi (chiếm 0.15%)


In [ ]:
missing_values(df_data)

##### 3.2.1) Xử lý giá trị mất mát của thuộc tính Embarked
---
Thuộc tính Embarked cho biết hành khách lên tàu ở Port nào? Embarked nhận 1 trong  3 giá trị S, Q, C.

Trong thuộc tính Embarked có 2 giá trị missing, để xử lý các giá trị missing này ta thực hiện bằng cách, lấy giá trị xuất hiện nhiều nhất của thuộc tính Embarked để thay thế cho giá trị missing.

Tổng 1309 hành khách, có 1307 hành khách có giá trị thuộc tính Embarked, 2 hành khách missing.

In [ ]:
#Dánh sách hành khách missing thộc tính Embarked trong tập dữ liệu
df_data[df_data['Embarked'].isnull()]

In [ ]:
#Thống kê số lượng hành khách theo cảng lên tàu:
df_data['Embarked'].value_counts()

In [ ]:
123/1307

Trong ba giá trị S, Q, C có số lần xuất hiện như sau:
* 0 (S): 914 lần (Xuất hiện nhiều nhất) chiếm ~70%  
* 1 (C) : 270 lần - chiếm 20.6%
* 2 (Q): 123 lần - chiếm 9.4%

==> Xử lý thuộc tính Embarked bằng giá trị xuất hiện nhiều nhất trong dữ liệu đó là cảng S

In [ ]:
df_data['Embarked'] = df_data['Embarked'].fillna('S')

In [ ]:
#Thống kê dữ liệu sau khi xử lý missing thuộc tính Embarked
df_data.isnull().sum()

#####  3.2.2) Xử lý dữ liệu mất mát của thuộc tính Age
---
Tập dữ liệu có 263 hành khách bị missing thông tin về tuổi chiếm tỷ lệ (20.09%)


In [ ]:
#Dánh sách hành khách missing thộc tính Embarked trong tập dữ liệu
df_data[df_data['Age'].isnull()]

**Có một sự tương quan giữa thuộc tính Age, Sex và Pclass.**

In [ ]:
grid = sns.FacetGrid(df_data, row='Pclass', col='Sex', size=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend()

**Bởi vậy chúng ta sẽ ước chừng giá trị Age tại các vị trí thiếu sử dụng các giá trị trung bình của thuộc tính Age theo các tập của sự kết hợp giữa thuộc tính Pclass và Sex. như, trung bình Age của Pclass = 1 và Sex = 0, Pclass = 1 và Sex = 1...vv.**

##### Thay thế các giá trị null bằng giá trị trung bình của độ tuổi theo Sex, Pclass
---

 Thay thế các giá trị null trong thuộc tính Age của các hành khách này dựa vào tuổi trung bình tương ứng với các hành khách có cùng chung thuộc tính Sex và Pclass với hành khách có tuổi bị missing đó. Bao gồm 6 nhóm
 
* Nhóm 1:  sex = Male, pclass = 1, (có độ tuổi trung bình 41)
 
* Nhóm 2: sex = Male, pclass = 2, (có độ tuổi trung bình 31)
 
* Nhóm 3: sex = Male, pclass = 3, (có độ tuổi trung bình 26)
 
* Nhóm 4: sex = Female, pclass = 1, (có độ tuổi trung bình 37)
 
* Nhóm 5: sex = Female, pclass = 2, (có độ tuổi trung bình 27)
 
* Nhóm 6: sex = Female, pclass = 3, (có độ tuổi trung bình 22)

In [ ]:
df_data.info()

In [ ]:
df_data.head()

In [ ]:
#Xử lý missing nhóm 1:
# Xác định tuổi trung bình của hành khách có thuộc tính sex = male, pclass = 1
#Danh sách các hành khách thỏa mãn:
df_male_1 =  df_data[(df_data['Sex']=='male') & (df_data['Pclass']==1)]


In [ ]:
Age_mean_male_1 = round(df_male_1['Age'].mean(),0)
print('Tuổi trung bình của nhóm hành khách có giới tính male và Pclass = 1:', Age_mean_male_1)

In [ ]:
#Thay thế tuổi của các hành khách có giới tính Sex=male và Pclass = 1 bị missing bằng tuổi trung bình của nhóm này
df_data[(df_data['Sex']=='male') & (df_data['Pclass']==1) & (df_data['Age'].isnull())]


In [ ]:
#Thực hiện điền tuổi trung bình vào các bảng ghi missing:
df_data.loc[(df_data['Sex']=='male') & (df_data['Pclass']==1) & (df_data['Age'].isnull()),\
            'Age'] = Age_mean_male_1

In [ ]:
#Xem lại dữ liệu sau xử lý:
df_data.iloc[[55,64,168,185,270],:]

**THỰC HIỆN TƯƠNG TỰ CHO CÁC NHÓM CÒN LẠI 2, 3, 4, 5, 6**

In [ ]:
#Nhóm 2:
df_male_2 =  df_data[(df_data['Sex']=='male') & (df_data['Pclass']==2)]
Age_mean_male_2 = round(df_male_2['Age'].mean(),0)
df_data.loc[(df_data['Sex']=='male') & (df_data['Pclass']==2) & (df_data['Age'].isnull()),\
            'Age'] = Age_mean_male_2

In [ ]:
print('Nhóm 2: Tuổi trung bình của nhóm hành khách có giới tính male và Pclass = 2:', Age_mean_male_2)

In [ ]:
#Nhóm 3:
df_male_3 =  df_data[(df_data['Sex']=='male') & (df_data['Pclass']==3)]
Age_mean_male_3 = round(df_male_3['Age'].mean(),0)
df_data.loc[(df_data['Sex']=='male') & (df_data['Pclass']==3) & (df_data['Age'].isnull()),\
            'Age'] = Age_mean_male_3

In [ ]:
print('Nhóm 3: Tuổi trung bình của nhóm hành khách có giới tính sex = male và Pclass = 3:', Age_mean_male_3)

In [ ]:
#Nhóm 4:
df_female_1 =  df_data[(df_data['Sex']=='female') & (df_data['Pclass']==1)]
Age_mean_female_1 = round(df_female_1['Age'].mean(),0)
df_data.loc[(df_data['Sex']=='female') & (df_data['Pclass']==1) & (df_data['Age'].isnull()),\
            'Age'] = Age_mean_female_1

In [ ]:
print('Nhóm 4: Tuổi trung bình của nhóm hành khách có giới tính sex = female và Pclass = 1:', Age_mean_female_1)

In [ ]:
#Nhóm 5:
df_female_2 =  df_data[(df_data['Sex']=='female') & (df_data['Pclass']==2)]
Age_mean_female_2 = round(df_female_2['Age'].mean(),0)
df_data.loc[(df_data['Sex']=='female') & (df_data['Pclass']==2) & (df_data['Age'].isnull()),\
            'Age'] = Age_mean_female_2

In [ ]:
print('Nhóm 5: Tuổi trung bình của nhóm hành khách có giới tính sex = female và Pclass = 2:', Age_mean_female_2)

In [ ]:
#Nhóm 6:
df_female_3 =  df_data[(df_data['Sex']=='female') & (df_data['Pclass']==3)]
Age_mean_female_3 = round(df_female_3['Age'].mean(),0)
df_data.loc[(df_data['Sex']=='female') & (df_data['Pclass']==3) & (df_data['Age'].isnull()),\
            'Age'] = Age_mean_female_3

In [ ]:
print('Nhóm 6: Tuổi trung bình của nhóm hành khách có giới tính sex = female và Pclass = 3:', Age_mean_female_3)

**DỮ LIỆU SAU KHI XỬ LÝ MISSING:**

In [ ]:
missing_values(df_data)

### 3.3) Rời rạc hóa dữ liệu
---

In [ ]:
#Tính tỷ lệ được cứu của hành khách theo từng khoảng tuổi:
n=5
df_temp = df_data.copy()
df_temp['AgeBand'] = pd.cut(df_temp['Age'], n)

df_temp[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

**Ta thấy rằng, tỷ lệ được cứu của hành khách có quan hệ theo độ tuổi, người già và trẻ em có tỷ lệ được cứu cao. Thực hiện chia dữ liệu độ tuổi theo từng đoạn**

* Dưới 16 tuổi     :   có tỷ lệ cứu sống cao nhất (52.2%)
* Từ 48 --> 64 tuổi:   có tỷ lệ cứu sống cao thứ nhì (42.5%)
* Từ 32 --> 48 tuổi:   có tỷ lệ cứu sống cao thứ ba (39.9%)
* Từ 16 --> 32 tuổi:   có tỷ lệ cứu sống cao thứ tư (34%)
* Trên 64 tuổi     :   có tỷ lệ cứu sống thấp nhất (15.4%)

In [ ]:
df_temp

**Thay thế tuổi tuyệt đối của hành khách bằng khoảng mà tuổi thuộc vào:**
* Dưới 16 tuổi        :   --> Age = 0
* Từ 16 --> 32 tuổi:   --> Age = 1
* Từ 32 --> 48 tuổi:   --> Age = 2
* Từ 48 --> 64 tuổi:   --> Age = 3
* Trên 64 tuổi        :   --> Age = 4

In [ ]:
df_data.loc[ df_data['Age'] <= 16, 'Age'] = 0
df_data.loc[(df_data['Age'] > 16) & (df_data['Age'] <= 32), 'Age'] = 1
df_data.loc[(df_data['Age'] > 32) & (df_data['Age'] <= 48), 'Age'] = 2
df_data.loc[(df_data['Age'] > 48) & (df_data['Age'] <= 64), 'Age'] = 3
df_data.loc[ df_data['Age'] > 64, 'Age'] = 4

In [ ]:
df_data

In [ ]:
#Ép Age sang kiểu dữ liệu số nguyên
df_data['Age'] = df_data['Age'].astype(int)
df_data.head()

### 3.4) Chuyển đổi dữ liệu thuộc tính sang dạng số
----
Một số thuộc tính của tập dữ liệu không phải dữ liệu số, trong bước này sẽ thực hiện chuyển đổi các dữ liệu Catagorical về dạng số để cho vào mô hình học máy.

Các thuộc tính cần chuyển đổi bao gồm:
* Sex 
* Embarked

In [ ]:
#Chuyển đổi thuộc tính Sex về dạng số nguyên (int)
# trong đó: Female = 1; Male = 0

df_data['Sex'] = df_data['Sex'].map( {'male': 0, 'female': 1} ).astype(int)

#Hiển thị dữ liệu 5 mẫu đầu tiên trong tập dữ liệu Train sau khi đã chuyển đổi.
df_data.head()

In [ ]:
#Chuyển đổi thuộc tính Embarked về dạng số nguyên (int)
# Trong đó: S = 0, C = 1, Q = 2
df_data['Embarked'] = df_data['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
df_data.head()

### 3.5) Lưu dữ liệu sau khi đã xử lý ra file .csv
---
Lưu dữ liệu sau khi đã chuẩn hóa ra file: Data_Titanic_ok.csv

In [ ]:
df_data.to_csv('Data_Titanic_ok.csv',index=None)

# THỰC HÀNH 4
---
**Mô tả tập dữ liệu:**

Tập dữ liệu Data_practice4 chứa dữ liệu của 961 bệnh nhân bị u vú.

Mỗi bản ghi tương ứng với một bệnh nhân, Bao gồm 6 thuộc tính:
1. PatienID: Thuộc tính cho biết mã số của bệnh nhân
2. Age: Thuộc tính cho biết tuổi của bệnh nhân, dữ liệu số [18-96 tuổi]
3. Shape: Thuộc tính cho biết hình dạng của khối u, bao gồm 4 giá trị: Round, Oval, Lobular, Irregular
4. Margin: Thuộc tính cho biết dạng đường biên của khối u, bao gồm 5 giá trị: Circumscribed, Microlobulated, Obscured, ill-defined, Spiculated
5. Density: Thuộc tính cho biết mật độ của khối u, bao gồm 4 giá trị: High, Iso, Low, Fat-containing
6. Target: Thuộc tính cho biết khối u là lành tính (0) - hay ác tính (1)

**Yêu cầu 4.1:**
Đọc tập dữ liệu Data_Practice4.csv vào biến DataFrame, hiển thị thông tin của biến, 5 bản ghi đầu tiên và thống kê dữ liệu.


In [ ]:
#{HỌC VIÊN VIẾT CODE VÀO ĐÂY!}








**Yêu cầu 4.2:**
Thông kê dữ liệu missing cho từng thuộc tính, và liệt kê ra các bản ghi bị missing tương ứng với thuộc tính đó.

In [ ]:
#{HỌC VIÊN VIẾT CODE VÀO ĐÂY!}








**Yêu cầu 4.3:** Phân tích, thống kê - xác định mức độ ảnh hưởng của các thuộc tính độc lập [Age, Shape, Margin, Density] tới thuộc tính phụ thuộc [Target] 

In [ ]:
#{HỌC VIÊN VIẾT CODE VÀO ĐÂY!}








**Yêu cầu 4.4:** Đề xuất và Áp dụng các kỹ chuẩn hóa, tiền xử lý dữ liệu phù hợp cho tập dữ liệu này để có thể đưa vào các mô hình học máy và lưu kết quả dữ liệu sau xử lý ra file Data_Practice4_ok.csv 


In [ ]:
#{HỌC VIÊN VIẾT CODE VÀO ĐÂY!}






